## OCIO and MaterialX

This notebook will cover one workflow of using OCIO to generate code for MaterialX.
We will be using the API based on OCIO 2.2 and above (released in October 2022). 

The aim of this notebook is to go over how OCIO can be used to generate "implementations".
The longer term MaterialX aim is to generate functional node graphs. This book will cover the setup required for generation, but can only show how source code implementation generation can be performed at the curren time.

The workflow covered is the "green" parts in this overall workflow diagram:
<img src="./images/OCIO_MaterialX_Workflow.png">

If / when graph generation is possible source code implementations can be swapped out for graph implementations.

The breakdown is as follows:
1. OCIO setup
2. Setting up OCIO configurations and getting available color spaces. 
4. Setting up OCIO  "processors" for generating color transforms
5. Generating source code implementations
6. Creating MaterialX implementations and definition wrappers for `color3` and `color4` variants.
7. Add definitions and implementations to the "standard" transform
library (`cmlib`)

For further OCI there is a fair bit of documentation available with a useful starting place [here](https://opencolorio.readthedocs.io/en/latest/guides/developing/developing.html)

### Setup

OpenColorIO is available as a pre-built Python package on PyPi [here](https://pypi.org/project/opencolorio/)

`pip` can be used to install the package which is called `PyOpenColorIO`

User can also build OpenColorIO by cloning the [GitHub repository](https://github.com/AcademySoftwareFoundation/OpenColorIO/).

For the purposes of generating color transform implementation, most of the build options can be disabled. For For building and installing the Python package, make sure that the appropriate build option is set (current `OCIO_BUILD_PYTHON`, `OCIO_INSTALL_EXT_PACKAGES` respectively).

An example set of options is given here:
```
-DOPENIMAGEIO_INCLUDE_DIR="" -DOPENIMAGEIO_LIBRARY="" -DOCIO_BUILD_DOCS=OFF -DBUILD_SHARED_LIBS=OFF -DOCIO_BUILD_TESTS=ON -DOCIO_BUILD_GPU_TESTS=OFF -DOCIO_BUILD_PYTHON=1 -DOCIO_BUILD_JAVA=0 -DOCIO_INSTALL_EXT_PACKAGES=ALL -DOCIO_BUILD_APPS=0 -DOCIO_BUILD_NUKE=0
```

A third alternative is to use a pre-built Python package which comes with another installation such as a DCC. A check should be made to not inadvertently use the incorrect version of the package.

For this book, we install from PyPi.

In [1]:
# Package install
#pip install OpenColorIO

Here we import PyOpenColorIO and MaterialX.

In [2]:
# Import OCIO package
import PyOpenColorIO as OCIO
import MaterialX as mx

print('OCIO version:', OCIO.GetVersion())
print('MaterialX version:', mx.getVersionString())

OCIO version: 2.5.0
MaterialX version: 1.39.5


### Configurations

As of version 2.2, `ACES Cg Config` and `ACES Studio Config` are packaged with `OCIO`, meaning that they are available to use without having to download them separately. The `getBuildInConfigs()` API is explained [here](https://opencolorio.readthedocs.io/en/latest/releases/ocio_2_2.html)

In [3]:
# Get the OCIO built in configs
registry = OCIO.BuiltinConfigRegistry().getBuiltinConfigs()

This items return canned be scanned and the appropriate configuration instantiated using `CreateFromBuiltInConfig()` In the following example we built a dictionary of configs along with the available color spaces.

In [4]:

# Create a dictionary of configs
configs = {}
for item in registry:
    # The short_name is the URI-style name.
    # The ui_name is the name to use in a user interface.
    short_name, ui_name, isRecommended, isDefault = item

    # Don't present built-in configs to users if they are no longer recommended.
    if isRecommended:
        # Create a config using the Cg config
        config = OCIO.Config.CreateFromBuiltinConfig(short_name)
        colorSpaces = None
        if config:
            colorSpaces = config.getColorSpaces()

        if colorSpaces:
            configs[short_name] = [config, colorSpaces]

# Print the configs
for config in configs:
    print('Built-in config:', config)
    csnames = configs[config][0].getColorSpaceNames()
    print('- Number of color spaces: %d' % len(csnames))
    #for csname in csnames:
    #    print('  -', csname)


Built-in config: cg-config-v4.0.0_aces-v2.0_ocio-v2.5
- Number of color spaces: 25
Built-in config: studio-config-v4.0.0_aces-v2.0_ocio-v2.5
- Number of color spaces: 55


A more direct way to get the desired config is to call `CreateFomFile` with the appropriate built in path. In this case we get the `ACES Cg Config`.`

In [5]:
acesCgConfigPath = 'ocio://cg-config-v1.0.0_aces-v1.3_ocio-v2.1'
builtinCfgC = OCIO.Config.CreateFromFile(acesCgConfigPath)
print('Built-in config:', builtinCfgC.getName())
csnames = builtinCfgC.getColorSpaceNames()
print('- Number of color spaces: %d' % len(csnames))

Built-in config: cg-config-v1.0.0_aces-v1.3_ocio-v2.1
- Number of color spaces: 14


### Color Spaces

To check what color space identifiers can be used we print out each color space name along with any aliases by calling `getAliases()` on each color space. 


In [6]:
from IPython.display import display_markdown

title = '| Configuration | Color Space | Aliases |\n'
title = title + '| --- | --- | --- |\n'

rows = ''
for c in configs:
    config = configs[c][0]
    colorSpaces = configs[c][1]
    for colorSpace in colorSpaces:
        aliases = colorSpace.getAliases()
        rows = rows + '| ' + c + ' | ' + colorSpace.getName() + ' | ' + ', '.join(aliases) + ' |\n'

md = '<details><summary>Color Spaces</summary>\n\n' + title + rows + '</details>'
display_markdown(md, raw=True)

<details><summary>Color Spaces</summary>

| Configuration | Color Space | Aliases |
| --- | --- | --- |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | sRGB - Display | srgb_display, srgb_rec709_display |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.2 Rec.709 - Display | g22_rec709_display |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Display P3 - Display | displayp3_display, srgb_p3d65_display |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Display P3 HDR - Display | displayp3_hdr_display, srgbe_p3d65_display |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | P3-D65 - Display | p3d65_display, g26_p3d65_display |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Rec.1886 Rec.709 - Display | rec1886_rec709_display, g24_rec709_display |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Rec.2100-PQ - Display | rec2100_pq_display, pq_rec2020_display |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | ST2084-P3-D65 - Display | st2084_p3d65_display, pq_p3d65_display |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | ACES2065-1 | aces2065_1, aces, ACES - ACES2065-1, lin_ap0, lin_ap0_scene |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | ACEScc | ACES - ACEScc, acescc_ap1, ocio:acescc_ap1_scene |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | ACEScct | ACES - ACEScct, acescct_ap1, ocio:acescct_ap1_scene |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | ACEScg | ACES - ACEScg, lin_ap1, lin_ap1_scene |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | sRGB Encoded Rec.709 (sRGB) | srgb_encoded_rec709_srgb, srgb_texture, srgb_rec709_scene, Utility - sRGB - Texture, Input - Generic - sRGB - Texture, sRGB - Texture, srgb_tx |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 1.8 Encoded Rec.709 | g18_encoded_rec709, g18_rec709, Utility - Gamma 1.8 - Rec.709 - Texture, Gamma 1.8 Rec.709 - Texture, g18_rec709_tx, g18_rec709_scene |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.2 Encoded Rec.709 | g22_encoded_rec709, g22_rec709, Utility - Gamma 2.2 - Rec.709 - Texture, Gamma 2.2 Rec.709 - Texture, g22_rec709_tx, g22_rec709_scene |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.4 Encoded Rec.709 | g24_encoded_rec709, g24_rec709, rec709_display, Utility - Rec.709 - Display, Gamma 2.4 Rec.709 - Texture, g24_rec709_tx, ocio:g24_rec709_scene |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | sRGB Encoded P3-D65 | srgb_encoded_p3d65, srgb_p3d65, srgb_displayp3, srgb_p3d65_scene, sRGB Encoded P3-D65 - Texture, srgb_encoded_p3d65_tx |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.2 Encoded AdobeRGB | g22_encoded_adobergb, adobergb, g22_adobergb_scene |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | sRGB Encoded AP1 | srgb_encoded_ap1, srgb_ap1, srgb_ap1_scene, sRGB Encoded AP1 - Texture, srgb_encoded_ap1_tx |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.2 Encoded AP1 | g22_encoded_ap1, g22_ap1, Gamma 2.2 AP1 - Texture, g22_ap1_tx, g22_ap1_scene |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear AdobeRGB | lin_adobergb, lin_adobergb_scene, Utility - Linear - Adobe RGB |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear P3-D65 | lin_p3d65, lin_displayp3, lin_p3d65_scene, Utility - Linear - P3-D65, Linear Display P3 |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear Rec.2020 | lin_rec2020, lin_rec2020_scene, Utility - Linear - Rec.2020 |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear Rec.709 (sRGB) | lin_rec709_srgb, lin_rec709, lin_rec709_scene, lin_srgb, Utility - Linear - sRGB, Utility - Linear - Rec.709 |
| cg-config-v4.0.0_aces-v2.0_ocio-v2.5 | Raw | Utility - Raw, none |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | sRGB - Display | srgb_display, srgb_rec709_display |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.2 Rec.709 - Display | g22_rec709_display |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Display P3 - Display | displayp3_display, srgb_p3d65_display |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Display P3 HDR - Display | displayp3_hdr_display, srgbe_p3d65_display |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | P3-D65 - Display | p3d65_display, g26_p3d65_display |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Rec.1886 Rec.709 - Display | rec1886_rec709_display, g24_rec709_display |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Rec.2100-HLG - Display | rec2100_hlg_display, hlg_rec2020_display |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Rec.2100-PQ - Display | rec2100_pq_display, pq_rec2020_display |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | ST2084-P3-D65 - Display | st2084_p3d65_display, pq_p3d65_display |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | ACES2065-1 | aces2065_1, aces, ACES - ACES2065-1, lin_ap0, lin_ap0_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | ACEScc | ACES - ACEScc, acescc_ap1, ocio:acescc_ap1_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | ACEScct | ACES - ACEScct, acescct_ap1, ocio:acescct_ap1_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | ACEScg | ACES - ACEScg, lin_ap1, lin_ap1_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | ADX10 | Input - ADX - ADX10, ocio:adx10_apd_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | ADX16 | Input - ADX - ADX16, ocio:adx16_apd_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Apple Log | apple_log, ocio:applelog_rec2020_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | ARRI LogC3 (EI800) | arri_logc3_ei800, logc3ei800_alexawide, Input - ARRI - V3 LogC (EI800) - Wide Gamut, ocio:arrilogc3_awg3_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear ARRI Wide Gamut 3 | lin_arri_wide_gamut_3, lin_alexawide, Input - ARRI - Linear - ALEXA Wide Gamut, ocio:lin_awg3_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | ARRI LogC4 | arri_logc4, ocio:arrilogc4_awg4_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear ARRI Wide Gamut 4 | lin_arri_wide_gamut_4, lin_awg4, ocio:lin_awg4_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | BMDFilm WideGamut Gen5 | bmdfilm_widegamut_gen5, ocio:bmdfilm5_wg5_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear BMD WideGamut Gen5 | lin_bmd_widegamut_gen5, ocio:lin_bmdwg5_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | DaVinci Intermediate WideGamut | davinci_intermediate_widegamut, ocio:davinci_dwg_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear DaVinci WideGamut | lin_davinci_widegamut, ocio:lin_dwg_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | CanonLog2 CinemaGamut D55 | canonlog2_cinemagamut_d55, canonlog2_cgamutday, Input - Canon - Canon-Log2 - Cinema Gamut Daylight, ocio:canonlog2_cgamutd55_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear CinemaGamut D55 | lin_cinemagamut_d55, lin_canoncgamutday, Input - Canon - Linear - Canon Cinema Gamut Daylight, ocio:lin_cgamutd55_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | CanonLog3 CinemaGamut D55 | canonlog3_cinemagamut_d55, canonlog3_cgamutday, Input - Canon - Canon-Log3 - Cinema Gamut Daylight, ocio:canonlog3_cgamutd55_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | D-Log D-Gamut | dlog_dgamut, ocio:djilog_dgamut_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear D-Gamut | lin_dgamut, ocio:lin_dgamut_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | V-Log V-Gamut | vlog_vgamut, Input - Panasonic - V-Log - V-Gamut, ocio:vlog_vgamut_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear V-Gamut | lin_vgamut, Input - Panasonic - Linear - V-Gamut, ocio:lin_vgamut_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Log3G10 REDWideGamutRGB | log3g10_redwidegamutrgb, rl3g10_rwg, Input - RED - REDLog3G10 - REDWideGamutRGB, ocio:redlog3g10_rwg_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear REDWideGamutRGB | lin_redwidegamutrgb, lin_rwg, Input - RED - Linear - REDWideGamutRGB, ocio:lin_rwg_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | S-Log3 S-Gamut3 | slog3_sgamut3, Input - Sony - S-Log3 - S-Gamut3, ocio:slog3_sgamut3_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | S-Log3 S-Gamut3.Cine | slog3_sgamut3cine, slog3_sgamutcine, Input - Sony - S-Log3 - S-Gamut3.Cine, ocio:slog3_sgamut3cine_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | S-Log3 Venice S-Gamut3 | slog3_venice_sgamut3, Input - Sony - S-Log3 - Venice S-Gamut3, ocio:slog3_sgamut3venice_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | S-Log3 Venice S-Gamut3.Cine | slog3_venice_sgamut3cine, slog3_venice_sgamutcine, Input - Sony - S-Log3 - Venice S-Gamut3.Cine, ocio:slog3_sgamut3cinevenice_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear S-Gamut3 | lin_sgamut3, Input - Sony - Linear - S-Gamut3, ocio:lin_sgamut3_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear S-Gamut3.Cine | lin_sgamut3cine, Input - Sony - Linear - S-Gamut3.Cine, ocio:lin_sgamut3cine_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear Venice S-Gamut3 | lin_venice_sgamut3, Input - Sony - Linear - Venice S-Gamut3, ocio:lin_sgamut3venice_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear Venice S-Gamut3.Cine | lin_venice_sgamut3cine, Input - Sony - Linear - Venice S-Gamut3.Cine, ocio:lin_sgamut3cinevenice_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | sRGB Encoded Rec.709 (sRGB) | srgb_encoded_rec709_srgb, srgb_texture, srgb_rec709_scene, Utility - sRGB - Texture, Input - Generic - sRGB - Texture, sRGB - Texture, srgb_tx |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 1.8 Encoded Rec.709 | g18_encoded_rec709, g18_rec709, Utility - Gamma 1.8 - Rec.709 - Texture, Gamma 1.8 Rec.709 - Texture, g18_rec709_tx, g18_rec709_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.2 Encoded Rec.709 | g22_encoded_rec709, g22_rec709, Utility - Gamma 2.2 - Rec.709 - Texture, Gamma 2.2 Rec.709 - Texture, g22_rec709_tx, g22_rec709_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.4 Encoded Rec.709 | g24_encoded_rec709, g24_rec709, rec709_display, Utility - Rec.709 - Display, Gamma 2.4 Rec.709 - Texture, g24_rec709_tx, ocio:g24_rec709_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Camera Rec.709 | camera_rec709, rec709_camera, Utility - Rec.709 - Camera, ocio:itu709_rec709_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | sRGB Encoded P3-D65 | srgb_encoded_p3d65, srgb_p3d65, srgb_displayp3, srgb_p3d65_scene, sRGB Encoded P3-D65 - Texture, srgb_encoded_p3d65_tx |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.2 Encoded AdobeRGB | g22_encoded_adobergb, adobergb, g22_adobergb_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | sRGB Encoded AP1 | srgb_encoded_ap1, srgb_ap1, srgb_ap1_scene, sRGB Encoded AP1 - Texture, srgb_encoded_ap1_tx |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Gamma 2.2 Encoded AP1 | g22_encoded_ap1, g22_ap1, Gamma 2.2 AP1 - Texture, g22_ap1_tx, g22_ap1_scene |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear AdobeRGB | lin_adobergb, lin_adobergb_scene, Utility - Linear - Adobe RGB |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear P3-D65 | lin_p3d65, lin_displayp3, lin_p3d65_scene, Utility - Linear - P3-D65, Linear Display P3 |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear Rec.2020 | lin_rec2020, lin_rec2020_scene, Utility - Linear - Rec.2020 |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Linear Rec.709 (sRGB) | lin_rec709_srgb, lin_rec709, lin_rec709_scene, lin_srgb, Utility - Linear - sRGB, Utility - Linear - Rec.709 |
| studio-config-v4.0.0_aces-v2.0_ocio-v2.5 | Raw | Utility - Raw, none |
</details>

### Supported Color Spaces in MaterialX

MaterialX currently uses color space names for :
1. Color space (`colorspace` attribute) tagging for input images (`filename` attributes) and colors (`color3` and `color4` types). 
2. Node category identifiers for node definitions of the form `<from color space>_<to color space name>` to specify color space conversion nodes. (Node definitions are support as of MaterialX 1.38.7)

Note that any valid color space name can be used for input tagging. 

At time of writing only specific color space conversions are supported via node definitions and hence can perform code injection during shader code generation.
Any color space information can still be passed as meta-data to the code generated (e.g. as is possible with the `OSL` generator).

Further note that only certain **aliases** which are valid MaterialX identifiers are recognized in this context. For example `g18_rec709` is used for color space `Gamma 1.8 Rec.709 - Texture` and `lin_rec709` is used for color space `Linear Rec.709 (sRGB)`.

### OCIO Shader Code Generation

It is possible to generate code color space transforms for certain code generation targets. 

This is done by:

1. Calling `getProcessor()` on the config with desired "source" and "destination" color spaces for the transform.
2. Creating a CPU or GPU processor 
3. Set the appropriate target language
4. Getting the shader code using `getShaderText()`

In [7]:
def generateShaderCode(config, sourceColorSpace, destColorSpace, language):
    cshaderCodee = ''
    if not config:
        return shaderCode

    # Create a processor for a pair of colorspaces (namely to go to linear)
    processor = None
    try:
        processor = config.getProcessor(sourceColorSpace, destColorSpace)
    except:
        return shaderCode

    gpuProcessor = None
    if processor:
        processor = processor.getOptimizedProcessor(OCIO.OPTIMIZATION_ALL) 
        gpuProcessor = processor.getDefaultGPUProcessor()
    if gpuProcessor:
        shaderDesc = OCIO.GpuShaderDesc.CreateShaderDesc()
        if shaderDesc:
            shaderDesc.setLanguage(language)
            gpuProcessor.extractGpuShaderInfo(shaderDesc)
            shaderCode = shaderDesc.getShaderText()
    
    return shaderCode

# Use GLSL as the shader language to produce, and linear as the target color space
language = OCIO.GpuLanguage.GPU_LANGUAGE_GLSL_4_0
targetColorSpace = 'lin_rec709'

# Go through all the config and create code for each transform

title = '| Source | Target | Code |\n'
title = title + '| --- | --- | --- |\n'

rows = ''
testedSources = set()
for c in configs:
    config = OCIO.Config.CreateFromBuiltinConfig(c)
    colorSpaces = config.getColorSpaces()
    for colorSpace in colorSpaces:
        colorSpaceName = colorSpace.getName()
        # Skip if the colorspace is already tested
        if colorSpaceName in testedSources:
            continue
        testedSources.add(colorSpaceName)

        code = generateShaderCode(config, colorSpace.getName(), targetColorSpace, language)
        code = code.replace('\n', '<br>')
        code = '<code>' + code + '</code>'
        rows = rows + '| ' + colorSpace.getName() + ' | ' + targetColorSpace + ' | ' + code + '|\n'

md = '<details><summary>Transform Code for GLSL</summary>\n\n' + title + rows + '</details>'
display_markdown(md, raw=True)

<details><summary>Transform Code for GLSL</summary>

| Source | Target | Code |
| --- | --- | --- |
| sRGB - Display | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'monCurveMirrorFwd' processing<br>  <br>  {<br>    vec4 breakPnt = vec4(0.0392857157, 0.0392857157, 0.0392857157, 1.);<br>    vec4 slope = vec4(0.077380158, 0.077380158, 0.077380158, 1.);<br>    vec4 scale = vec4(0.947867274, 0.947867274, 0.947867274, 0.999998987);<br>    vec4 offset = vec4(0.0521326996, 0.0521326996, 0.0521326996, 9.99998974e-07);<br>    vec4 gamma = vec4(2.4000001, 2.4000001, 2.4000001, 1.00000095);<br>    vec4 signcol = sign(outColor);;<br>    outColor = abs( outColor );<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 linSeg = outColor * slope;<br>    vec4 powSeg = pow( scale * outColor + offset, gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * linSeg;<br>    res = signcol * res;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Gamma 2.2 Rec.709 - Display | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'basicMirrorFwd' processing<br>  <br>  {<br>    vec4 gamma = vec4(2.2000000000000002, 2.2000000000000002, 2.2000000000000002, 1.);<br>    vec4 signcol = sign(outColor);;<br>    vec4 res = signcol * pow( abs( outColor ), gamma );<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Display P3 - Display | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'monCurveMirrorFwd' processing<br>  <br>  {<br>    vec4 breakPnt = vec4(0.0392857157, 0.0392857157, 0.0392857157, 1.);<br>    vec4 slope = vec4(0.077380158, 0.077380158, 0.077380158, 1.);<br>    vec4 scale = vec4(0.947867274, 0.947867274, 0.947867274, 0.999998987);<br>    vec4 offset = vec4(0.0521326996, 0.0521326996, 0.0521326996, 9.99998974e-07);<br>    vec4 gamma = vec4(2.4000001, 2.4000001, 2.4000001, 1.00000095);<br>    vec4 signcol = sign(outColor);;<br>    outColor = abs( outColor );<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 linSeg = outColor * slope;<br>    vec4 powSeg = pow( scale * outColor + offset, gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * linSeg;<br>    res = signcol * res;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.2249401762805603, -0.04205695470968808, -0.019637554590334519, 0., -0.2249401762805579, 1.0420569547096905, -0.07863604555063225, 0., -0., 0., 1.0982736001409619, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Display P3 HDR - Display | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'monCurveMirrorFwd' processing<br>  <br>  {<br>    vec4 breakPnt = vec4(0.0392857157, 0.0392857157, 0.0392857157, 1.);<br>    vec4 slope = vec4(0.077380158, 0.077380158, 0.077380158, 1.);<br>    vec4 scale = vec4(0.947867274, 0.947867274, 0.947867274, 0.999998987);<br>    vec4 offset = vec4(0.0521326996, 0.0521326996, 0.0521326996, 9.99998974e-07);<br>    vec4 gamma = vec4(2.4000001, 2.4000001, 2.4000001, 1.00000095);<br>    vec4 signcol = sign(outColor);;<br>    outColor = abs( outColor );<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 linSeg = outColor * slope;<br>    vec4 powSeg = pow( scale * outColor + offset, gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * linSeg;<br>    res = signcol * res;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.2249401762805603, -0.04205695470968808, -0.019637554590334519, 0., -0.2249401762805579, 1.0420569547096905, -0.07863604555063225, 0., -0., 0., 1.0982736001409619, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| P3-D65 - Display | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'basicMirrorFwd' processing<br>  <br>  {<br>    vec4 gamma = vec4(2.6000000000000001, 2.6000000000000001, 2.6000000000000001, 1.);<br>    vec4 signcol = sign(outColor);;<br>    vec4 res = signcol * pow( abs( outColor ), gamma );<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.2249401762805603, -0.04205695470968808, -0.019637554590334519, 0., -0.2249401762805579, 1.0420569547096905, -0.07863604555063225, 0., -0., 0., 1.0982736001409619, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Rec.1886 Rec.709 - Display | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'basicMirrorFwd' processing<br>  <br>  {<br>    vec4 gamma = vec4(2.3999999999999999, 2.3999999999999999, 2.3999999999999999, 1.);<br>    vec4 signcol = sign(outColor);;<br>    vec4 res = signcol * pow( abs( outColor ), gamma );<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Rec.2100-PQ - Display | lin_rec709 | <code><br>// Declaration of all textures<br><br>uniform sampler2D ocio_lut1d_0Sampler;<br><br>// Declaration of all helper methods<br><br>vec2 ocio_lut1d_0_computePos(float f)<br>{<br>  float dep;<br>  float abs_f = abs(f);<br>  if (abs_f > 6.10351562e-05)<br>  {<br>    vec3 fComp = vec3(15., 15., 15.);<br>    float absarr = min( abs_f, 65504.);<br>    fComp.x = floor( log2( absarr ) );<br>    float lower = pow( 2.0, fComp.x );<br>    fComp.y = ( absarr - lower ) / lower;<br>    vec3 scale = vec3(1024., 1024., 1024.);<br>    dep = dot( fComp, scale );<br>  }<br>  else<br>  {<br>    dep = abs_f * 16777216.;<br>  }<br>  dep += (f < 0.) ? 32768.0 : 0.0;<br>  vec2 retVal;<br>  retVal.y = floor(dep / 4095.);<br>  retVal.x = dep - retVal.y * 4095.;<br>  retVal.x = (retVal.x + 0.5) / 4096.;<br>  retVal.y = (retVal.y + 0.5) / 17.;<br>  return retVal;<br>}<br><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add LUT 1D processing for ocio_lut1d_0<br>  <br>  {<br>    outColor.r = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.r)).r;<br>    outColor.g = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.g)).r;<br>    outColor.b = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.b)).r;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6604910021084351, -0.12455047452159074, -0.01815076335490531, 0., -0.58764113878854807, 1.132899897125963, -0.1005788980080078, 0., -0.072849863319885827, -0.0083494226043691246, 1.1187296613629085, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| ST2084-P3-D65 - Display | lin_rec709 | <code><br>// Declaration of all textures<br><br>uniform sampler2D ocio_lut1d_0Sampler;<br><br>// Declaration of all helper methods<br><br>vec2 ocio_lut1d_0_computePos(float f)<br>{<br>  float dep;<br>  float abs_f = abs(f);<br>  if (abs_f > 6.10351562e-05)<br>  {<br>    vec3 fComp = vec3(15., 15., 15.);<br>    float absarr = min( abs_f, 65504.);<br>    fComp.x = floor( log2( absarr ) );<br>    float lower = pow( 2.0, fComp.x );<br>    fComp.y = ( absarr - lower ) / lower;<br>    vec3 scale = vec3(1024., 1024., 1024.);<br>    dep = dot( fComp, scale );<br>  }<br>  else<br>  {<br>    dep = abs_f * 16777216.;<br>  }<br>  dep += (f < 0.) ? 32768.0 : 0.0;<br>  vec2 retVal;<br>  retVal.y = floor(dep / 4095.);<br>  retVal.x = dep - retVal.y * 4095.;<br>  retVal.x = (retVal.x + 0.5) / 4096.;<br>  retVal.y = (retVal.y + 0.5) / 17.;<br>  return retVal;<br>}<br><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add LUT 1D processing for ocio_lut1d_0<br>  <br>  {<br>    outColor.r = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.r)).r;<br>    outColor.g = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.g)).r;<br>    outColor.b = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.b)).r;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.2249401762805603, -0.04205695470968808, -0.019637554590334519, 0., -0.2249401762805579, 1.0420569547096905, -0.07863604555063225, 0., -0., 0., 1.0982736001409619, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| ACES2065-1 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(2.5216861867438798, -0.27647991422992202, -0.015378064966034201, 0., -1.1341309882397199, 1.37271908766826, -0.152975335867399, 0., -0.38755519850416398, -0.096239173438334005, 1.16835340083343, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| ACEScc | lin_rec709 | <code><br>// Declaration of all textures<br><br>uniform sampler2D ocio_lut1d_0Sampler;<br><br>// Declaration of all helper methods<br><br>vec2 ocio_lut1d_0_computePos(float f)<br>{<br>  float dep = clamp(f, 0.0, 1.0) * 4095.;<br>  vec2 retVal;<br>  retVal.y = floor(dep / 4095.);<br>  retVal.x = dep - retVal.y * 4095.;<br>  retVal.x = (retVal.x + 0.5) / 4096.;<br>  retVal.y = (retVal.y + 0.5) / 2.;<br>  return retVal;<br>}<br><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Range processing<br>  <br>  {<br>    outColor.rgb = outColor.rgb * vec3(0.53763440860215062, 0.53763440860215062, 0.53763440860215062) + vec3(0.19354838709677422, 0.19354838709677422, 0.19354838709677422);<br>    outColor.rgb = max(vec3(0., 0., 0.), outColor.rgb);<br>    outColor.rgb = min(vec3(1., 1., 1.), outColor.rgb);<br>  }<br>  <br>  // Add LUT 1D processing for ocio_lut1d_0<br>  <br>  {<br>    outColor.r = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.r)).r;<br>    outColor.g = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.g)).r;<br>    outColor.b = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.b)).r;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(0.69545224135745187, 0.044794563372037723, -0.0055258825581135443, 0., 0.14067869647029416, 0.85967111845642175, 0.0040252103059786595, 0., 0.16386906217225405, 0.0955343181715404, 1.0015006722521349, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add Range processing<br>  <br>  {<br>    outColor.rgb = max(vec3(0., 0., 0.), outColor.rgb);<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(2.5216861867438798, -0.27647991422992202, -0.015378064966034201, 0., -1.1341309882397199, 1.37271908766826, -0.152975335867399, 0., -0.38755519850416398, -0.096239173438334005, 1.16835340083343, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| ACEScct | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.155251175, 0.155251175, 0.155251175);<br>    vec3 linear_segment_offset = vec3(0.0729055703, 0.0729055703, 0.0729055703);<br>    vec3 linear_segment_slopeinv = vec3(0.0948745236, 0.0948745236, 0.0948745236);<br>    vec3 lin_slopeinv = vec3(1., 1., 1.);<br>    vec3 lin_offset = vec3(0., 0., 0.);<br>    vec3 log_slopeinv = vec3(17.5200005, 17.5200005, 17.5200005);<br>    vec3 log_base = vec3(2., 2., 2.);<br>    vec3 log_offset = vec3(0.5547945205479452, 0.5547945205479452, 0.5547945205479452);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.7050509926579815, -0.1302564175070435, -0.024003356804618046, 0., -0.62179212065700573, 1.140804736575405, -0.12896897606497093, 0., -0.083258872000979672, -0.010548319068357662, 1.1529723328695858, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| ACEScg | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.7050509926579815, -0.1302564175070435, -0.024003356804618046, 0., -0.62179212065700573, 1.140804736575405, -0.12896897606497093, 0., -0.083258872000979672, -0.010548319068357662, 1.1529723328695858, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| sRGB Encoded Rec.709 (sRGB) | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'monCurveFwd' processing<br>  <br>  {<br>    vec4 breakPnt = vec4(0.0392857157, 0.0392857157, 0.0392857157, 1.);<br>    vec4 slope = vec4(0.077380158, 0.077380158, 0.077380158, 1.);<br>    vec4 scale = vec4(0.947867274, 0.947867274, 0.947867274, 0.999998987);<br>    vec4 offset = vec4(0.0521326996, 0.0521326996, 0.0521326996, 9.99998974e-07);<br>    vec4 gamma = vec4(2.4000001, 2.4000001, 2.4000001, 1.00000095);<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 linSeg = outColor * slope;<br>    vec4 powSeg = pow( max( vec4(0., 0., 0., 0.), scale * outColor + offset), gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * linSeg;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Gamma 1.8 Encoded Rec.709 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'basicPassThruFwd' processing<br>  <br>  {<br>    vec4 gamma = vec4(1.8, 1.8, 1.8, 1.);<br>    vec4 breakPnt = vec4(0., 0., 0., 0.);<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 powSeg = pow(max( vec4(0., 0., 0., 0.), outColor ), gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * outColor;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Gamma 2.2 Encoded Rec.709 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'basicPassThruFwd' processing<br>  <br>  {<br>    vec4 gamma = vec4(2.2000000000000002, 2.2000000000000002, 2.2000000000000002, 1.);<br>    vec4 breakPnt = vec4(0., 0., 0., 0.);<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 powSeg = pow(max( vec4(0., 0., 0., 0.), outColor ), gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * outColor;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Gamma 2.4 Encoded Rec.709 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'basicPassThruFwd' processing<br>  <br>  {<br>    vec4 gamma = vec4(2.3999999999999999, 2.3999999999999999, 2.3999999999999999, 1.);<br>    vec4 breakPnt = vec4(0., 0., 0., 0.);<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 powSeg = pow(max( vec4(0., 0., 0., 0.), outColor ), gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * outColor;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| sRGB Encoded P3-D65 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'monCurveFwd' processing<br>  <br>  {<br>    vec4 breakPnt = vec4(0.0392857157, 0.0392857157, 0.0392857157, 1.);<br>    vec4 slope = vec4(0.077380158, 0.077380158, 0.077380158, 1.);<br>    vec4 scale = vec4(0.947867274, 0.947867274, 0.947867274, 0.999998987);<br>    vec4 offset = vec4(0.0521326996, 0.0521326996, 0.0521326996, 9.99998974e-07);<br>    vec4 gamma = vec4(2.4000001, 2.4000001, 2.4000001, 1.00000095);<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 linSeg = outColor * slope;<br>    vec4 powSeg = pow( max( vec4(0., 0., 0., 0.), scale * outColor + offset), gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * linSeg;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.2249401762805545, -0.042056954709687636, -0.019637554590334446, 0., -0.22494017628056287, 1.0420569547096903, -0.078636045550631917, 0., -0., 0., 1.0982736001409681, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Gamma 2.2 Encoded AdobeRGB | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'basicPassThruFwd' processing<br>  <br>  {<br>    vec4 gamma = vec4(2.19921875, 2.19921875, 2.19921875, 1.);<br>    vec4 breakPnt = vec4(0., 0., 0., 0.);<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 powSeg = pow(max( vec4(0., 0., 0., 0.), outColor ), gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * outColor;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.3983557439607741, -0., -0., 0., -0.39835574396077833, 1., -0.042928989294473863, 0., -0., -0., 1.0429289892944664, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| sRGB Encoded AP1 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'monCurveFwd' processing<br>  <br>  {<br>    vec4 breakPnt = vec4(0.0392857157, 0.0392857157, 0.0392857157, 1.);<br>    vec4 slope = vec4(0.077380158, 0.077380158, 0.077380158, 1.);<br>    vec4 scale = vec4(0.947867274, 0.947867274, 0.947867274, 0.999998987);<br>    vec4 offset = vec4(0.0521326996, 0.0521326996, 0.0521326996, 9.99998974e-07);<br>    vec4 gamma = vec4(2.4000001, 2.4000001, 2.4000001, 1.00000095);<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 linSeg = outColor * slope;<br>    vec4 powSeg = pow( max( vec4(0., 0., 0., 0.), scale * outColor + offset), gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * linSeg;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.7050509926579756, -0.13025641750704287, -0.02400335680461799, 0., -0.62179212065700873, 1.1408047365754082, -0.12896897606497126, 0., -0.083258872000981712, -0.010548319068357213, 1.152972332869586, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Gamma 2.2 Encoded AP1 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'basicPassThruFwd' processing<br>  <br>  {<br>    vec4 gamma = vec4(2.2000000000000002, 2.2000000000000002, 2.2000000000000002, 1.);<br>    vec4 breakPnt = vec4(0., 0., 0., 0.);<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 powSeg = pow(max( vec4(0., 0., 0., 0.), outColor ), gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * outColor;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.7050509926579756, -0.13025641750704287, -0.02400335680461799, 0., -0.62179212065700873, 1.1408047365754082, -0.12896897606497126, 0., -0.083258872000981712, -0.010548319068357213, 1.152972332869586, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear AdobeRGB | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.3983557439607741, -0., -0., 0., -0.39835574396077833, 1., -0.042928989294473863, 0., -0., -0., 1.0429289892944664, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear P3-D65 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.2249401762805545, -0.042056954709687636, -0.019637554590334446, 0., -0.22494017628056287, 1.0420569547096903, -0.078636045550631917, 0., -0., 0., 1.0982736001409681, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear Rec.2020 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6604910021084354, -0.12455047452159097, -0.018150763354905279, 0., -0.58764113878854762, 1.1328998971259598, -0.10057889800800768, 0., -0.072849863319883981, -0.0083494226043701082, 1.118729661362905, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear Rec.709 (sRGB) | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br><br>  return outColor;<br>}<br></code>|
| Raw | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br><br>  return outColor;<br>}<br></code>|
| Rec.2100-HLG - Display | lin_rec709 | <code><br>// Declaration of all textures<br><br>uniform sampler2D ocio_lut1d_0Sampler;<br><br>// Declaration of all helper methods<br><br>vec2 ocio_lut1d_0_computePos(float f)<br>{<br>  float dep;<br>  float abs_f = abs(f);<br>  if (abs_f > 6.10351562e-05)<br>  {<br>    vec3 fComp = vec3(15., 15., 15.);<br>    float absarr = min( abs_f, 65504.);<br>    fComp.x = floor( log2( absarr ) );<br>    float lower = pow( 2.0, fComp.x );<br>    fComp.y = ( absarr - lower ) / lower;<br>    vec3 scale = vec3(1024., 1024., 1024.);<br>    dep = dot( fComp, scale );<br>  }<br>  else<br>  {<br>    dep = abs_f * 16777216.;<br>  }<br>  dep += (f < 0.) ? 32768.0 : 0.0;<br>  vec2 retVal;<br>  retVal.y = floor(dep / 4095.);<br>  retVal.x = dep - retVal.y * 4095.;<br>  retVal.x = (retVal.x + 0.5) / 4096.;<br>  retVal.y = (retVal.y + 0.5) / 17.;<br>  return retVal;<br>}<br><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add LUT 1D processing for ocio_lut1d_0<br>  <br>  {<br>    outColor.r = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.r)).r;<br>    outColor.g = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.g)).r;<br>    outColor.b = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.b)).r;<br>  }<br>  <br>  // Add FixedFunction 'REC2100_Surround (Inverse)' processing<br>  <br>  {<br>    float Y = 0.2627 * outColor.rgb.r + 0.6780 * outColor.rgb.g + 0.0593 * outColor.rgb.b;<br>    Y = max( 0.000464158948, abs(Y) );<br>    float Ypow_over_Y = pow( Y, 0.200000048);<br>    outColor.rgb = outColor.rgb * Ypow_over_Y;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(4.4431504677377864, -0.33327280811813209, -0.048567907075523657, 0., -1.572413218352265, 3.0314194424563343, -0.26912953889023716, 0., -0.19493204351808163, -0.022341428470758172, 2.9935026518331842, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| ADX10 | lin_rec709 | <code><br>// Declaration of all textures<br><br>uniform sampler2D ocio_lut1d_0Sampler;<br><br>// Declaration of all helper methods<br><br>vec2 ocio_lut1d_0_computePos(float f)<br>{<br>  float dep;<br>  float abs_f = abs(f);<br>  if (abs_f > 6.10351562e-05)<br>  {<br>    vec3 fComp = vec3(15., 15., 15.);<br>    float absarr = min( abs_f, 65504.);<br>    fComp.x = floor( log2( absarr ) );<br>    float lower = pow( 2.0, fComp.x );<br>    fComp.y = ( absarr - lower ) / lower;<br>    vec3 scale = vec3(1024., 1024., 1024.);<br>    dep = dot( fComp, scale );<br>  }<br>  else<br>  {<br>    dep = abs_f * 16777216.;<br>  }<br>  dep += (f < 0.) ? 32768.0 : 0.0;<br>  vec2 retVal;<br>  retVal.y = floor(dep / 4095.);<br>  retVal.x = dep - retVal.y * 4095.;<br>  retVal.x = (retVal.x + 0.5) / 4096.;<br>  retVal.y = (retVal.y + 0.5) / 17.;<br>  return retVal;<br>}<br><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.5462235799999999, 0.12073445999999999, 0.33010163999999997, 0., 0.45415061999999995, 1.9831468799999998, 0.15152675999999998, 0., 0.045625799999999994, -0.057881339999999996, 1.5643715999999999, 0., 0., 0., 0., 1.) * tmp;<br>    res = vec4(-0.189999998, -0.189999998, -0.189999998, 0.) + res;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add LUT 1D processing for ocio_lut1d_0<br>  <br>  {<br>    outColor.r = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.r)).r;<br>    outColor.g = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.g)).r;<br>    outColor.b = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.b)).r;<br>  }<br>  <br>  // Add Log 'Anti-Log' processing<br>  <br>  {<br>    outColor.rgb = pow( vec3(10., 10., 10.), outColor.rgb);<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6820732265392047, -0.037560306982519782, -0.012683034306924419, 0., -0.58002116166042417, 1.0061809359326015, -0.022886076957909493, 0., -0.10204818932679967, 0.031380333441656777, 1.0355574277308224, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| ADX16 | lin_rec709 | <code><br>// Declaration of all textures<br><br>uniform sampler2D ocio_lut1d_0Sampler;<br><br>// Declaration of all helper methods<br><br>vec2 ocio_lut1d_0_computePos(float f)<br>{<br>  float dep;<br>  float abs_f = abs(f);<br>  if (abs_f > 6.10351562e-05)<br>  {<br>    vec3 fComp = vec3(15., 15., 15.);<br>    float absarr = min( abs_f, 65504.);<br>    fComp.x = floor( log2( absarr ) );<br>    float lower = pow( 2.0, fComp.x );<br>    fComp.y = ( absarr - lower ) / lower;<br>    vec3 scale = vec3(1024., 1024., 1024.);<br>    dep = dot( fComp, scale );<br>  }<br>  else<br>  {<br>    dep = abs_f * 16777216.;<br>  }<br>  dep += (f < 0.) ? 32768.0 : 0.0;<br>  vec2 retVal;<br>  retVal.y = floor(dep / 4095.);<br>  retVal.x = dep - retVal.y * 4095.;<br>  retVal.x = (retVal.x + 0.5) / 4096.;<br>  retVal.y = (retVal.y + 0.5) / 17.;<br>  return retVal;<br>}<br><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(6.19084569375, 0.48340254374999997, 1.3216771125, 0., 1.8183504937499999, 7.9402206, 0.60669026250000002, 0., 0.18267881249999998, -0.23174814374999997, 6.263507624999999, 0., 0., 0., 0., 1.) * tmp;<br>    res = vec4(-0.189999998, -0.189999998, -0.189999998, 0.) + res;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br>  <br>  // Add LUT 1D processing for ocio_lut1d_0<br>  <br>  {<br>    outColor.r = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.r)).r;<br>    outColor.g = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.g)).r;<br>    outColor.b = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.b)).r;<br>  }<br>  <br>  // Add Log 'Anti-Log' processing<br>  <br>  {<br>    outColor.rgb = pow( vec3(10., 10., 10.), outColor.rgb);<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6820732265392047, -0.037560306982519782, -0.012683034306924419, 0., -0.58002116166042417, 1.0061809359326015, -0.022886076957909493, 0., -0.10204818932679967, 0.031380333441656777, 1.0355574277308224, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Apple Log | lin_rec709 | <code><br>// Declaration of all textures<br><br>uniform sampler2D ocio_lut1d_0Sampler;<br><br>// Declaration of all helper methods<br><br>vec2 ocio_lut1d_0_computePos(float f)<br>{<br>  float dep;<br>  float abs_f = abs(f);<br>  if (abs_f > 6.10351562e-05)<br>  {<br>    vec3 fComp = vec3(15., 15., 15.);<br>    float absarr = min( abs_f, 65504.);<br>    fComp.x = floor( log2( absarr ) );<br>    float lower = pow( 2.0, fComp.x );<br>    fComp.y = ( absarr - lower ) / lower;<br>    vec3 scale = vec3(1024., 1024., 1024.);<br>    dep = dot( fComp, scale );<br>  }<br>  else<br>  {<br>    dep = abs_f * 16777216.;<br>  }<br>  dep += (f < 0.) ? 32768.0 : 0.0;<br>  vec2 retVal;<br>  retVal.y = floor(dep / 4095.);<br>  retVal.x = dep - retVal.y * 4095.;<br>  retVal.x = (retVal.x + 0.5) / 4096.;<br>  retVal.y = (retVal.y + 0.5) / 17.;<br>  return retVal;<br>}<br><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add LUT 1D processing for ocio_lut1d_0<br>  <br>  {<br>    outColor.r = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.r)).r;<br>    outColor.g = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.g)).r;<br>    outColor.b = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.b)).r;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.660491002108432, -0.12455047452159082, -0.018150763354905224, 0., -0.58764113878855107, 1.1328998971259643, -0.10057889800800789, 0., -0.072849863319885869, -0.0083494226043691055, 1.1187296613629099, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| ARRI LogC3 (EI800) | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.149658144, 0.149658144, 0.149658144);<br>    vec3 linear_segment_offset = vec3(0.0928093642, 0.0928093642, 0.0928093642);<br>    vec3 linear_segment_slopeinv = vec3(0.186301097, 0.186301097, 0.186301097);<br>    vec3 lin_slopeinv = vec3(0.180000007, 0.180000007, 0.180000007);<br>    vec3 lin_offset = vec3(0.052272275025168798, 0.052272275025168798, 0.052272275025168798);<br>    vec3 log_slopeinv = vec3(4.04547691, 4.04547691, 4.04547691);<br>    vec3 log_base = vec3(10., 10., 10.);<br>    vec3 log_offset = vec3(0.38553699869244301, 0.38553699869244301, 0.38553699869244301);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6175960353244454, -0.071010304591282061, -0.021123885575728883, 0., -0.53423642434848773, 1.3374356321770418, -0.23237438347978714, 0., -0.083359610975958662, -0.26642532758575715, 1.253498269055507, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear ARRI Wide Gamut 3 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6175960353244454, -0.071010304591282061, -0.021123885575728883, 0., -0.53423642434848773, 1.3374356321770418, -0.23237438347978714, 0., -0.083359610975958662, -0.26642532758575715, 1.253498269055507, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| ARRI LogC4 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0., 0., 0.);<br>    vec3 linear_segment_offset = vec3(0.158956334, 0.158956334, 0.158956334);<br>    vec3 linear_segment_slopeinv = vec3(0.113597214, 0.113597214, 0.113597214);<br>    vec3 lin_slopeinv = vec3(0.000448063511, 0.000448063511, 0.000448063511);<br>    vec3 lin_offset = vec3(64., 64., 64.);<br>    vec3 log_slopeinv = vec3(15.4331894, 15.4331894, 15.4331894);<br>    vec3 log_base = vec3(2., 2., 2.);<br>    vec3 log_offset = vec3(-0.29590839268258601, -0.29590839268258601, -0.29590839268258601);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.8929404968219639, -0.20644836083723977, -0.012258112702638239, 0., -0.77800083137239318, 1.3430260992709093, -0.15732590776965169, 0., -0.11493966544957079, -0.13657773843366997, 1.1695840204722867, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear ARRI Wide Gamut 4 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.8929404968219639, -0.20644836083723977, -0.012258112702638239, 0., -0.77800083137239318, 1.3430260992709093, -0.15732590776965169, 0., -0.11493966544957079, -0.13657773843366997, 1.1695840204722867, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| BMDFilm WideGamut Gen5 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.133883744, 0.133883744, 0.133883744);<br>    vec3 linear_segment_offset = vec3(0.0924657211, 0.0924657211, 0.0924657211);<br>    vec3 linear_segment_slopeinv = vec3(0.120720379, 0.120720379, 0.120720379);<br>    vec3 lin_slopeinv = vec3(1., 1., 1.);<br>    vec3 lin_offset = vec3(0.0054940724322578103, 0.0054940724322578103, 0.0054940724322578103);<br>    vec3 log_slopeinv = vec3(11.5036726, 11.5036726, 11.5036726);<br>    vec3 log_base = vec3(2.71828182845905, 2.71828182845905, 2.71828182845905);<br>    vec3 log_offset = vec3(0.53001333922919402, 0.53001333922919402, 0.53001333922919402);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.5685053307142094, -0.086765930966416174, -0.05212011917108085, 0., -0.51955620578769812, 1.3477854013561954, -0.25469373659499694, 0., -0.048949124926515208, -0.26101947038977646, 1.3068138557660767, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear BMD WideGamut Gen5 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.5685053307142094, -0.086765930966416174, -0.05212011917108085, 0., -0.51955620578769812, 1.3477854013561954, -0.25469373659499694, 0., -0.048949124926515208, -0.26101947038977646, 1.3068138557660767, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| DaVinci Intermediate WideGamut | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.0274066925, 0.0274066925, 0.0274066925);<br>    vec3 linear_segment_offset = vec3(-7.95820743e-09, -7.95820743e-09, -7.95820743e-09);<br>    vec3 linear_segment_slopeinv = vec3(0.0957462937, 0.0957462937, 0.0957462937);<br>    vec3 lin_slopeinv = vec3(1., 1., 1.);<br>    vec3 lin_offset = vec3(0.0074999999999999997, 0.0074999999999999997, 0.0074999999999999997);<br>    vec3 log_slopeinv = vec3(13.6439648, 13.6439648, 13.6439648);<br>    vec3 log_base = vec3(2., 2., 2.);<br>    vec3 log_offset = vec3(0.51304735999999995, 0.51304735999999995, 0.51304735999999995);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.8987312079363696, -0.1694642609483365, -0.12161393715142486, 0., -0.78866341542508211, 1.4922628969176159, -0.32192612718818653, 0., -0.11006779251129152, -0.32279863596927649, 1.4435400643396084, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear DaVinci WideGamut | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.8987312079363696, -0.1694642609483365, -0.12161393715142486, 0., -0.78866341542508211, 1.4922628969176159, -0.32192612718818653, 0., -0.11006779251129152, -0.32279863596927649, 1.4435400643396084, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| CanonLog2 CinemaGamut D55 | lin_rec709 | <code><br>// Declaration of all textures<br><br>uniform sampler2D ocio_lut1d_0Sampler;<br><br>// Declaration of all helper methods<br><br>vec2 ocio_lut1d_0_computePos(float f)<br>{<br>  float dep = clamp(f, 0.0, 1.0) * 4095.;<br>  vec2 retVal;<br>  retVal.y = floor(dep / 4095.);<br>  retVal.x = dep - retVal.y * 4095.;<br>  retVal.x = (retVal.x + 0.5) / 4096.;<br>  retVal.y = (retVal.y + 0.5) / 2.;<br>  return retVal;<br>}<br><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add LUT 1D processing for ocio_lut1d_0<br>  <br>  {<br>    outColor.r = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.r)).r;<br>    outColor.g = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.g)).r;<br>    outColor.b = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.b)).r;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.9237070997046666, -0.20504593605401519, -0.023285583594355667, 0., -0.79501788145104413, 1.499361314350298, -0.42677817014753039, 0., -0.12868921825362739, -0.29431537829627863, 1.4500637537418828, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear CinemaGamut D55 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.9237070997046672, -0.20504593605401522, -0.023285583594355678, 0., -0.79501788145105079, 1.4993613143503051, -0.42677817014753106, 0., -0.12868921825362856, -0.29431537829627935, 1.4500637537418875, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| CanonLog3 CinemaGamut D55 | lin_rec709 | <code><br>// Declaration of all textures<br><br>uniform sampler2D ocio_lut1d_0Sampler;<br><br>// Declaration of all helper methods<br><br>vec2 ocio_lut1d_0_computePos(float f)<br>{<br>  float dep = clamp(f, 0.0, 1.0) * 4095.;<br>  vec2 retVal;<br>  retVal.y = floor(dep / 4095.);<br>  retVal.x = dep - retVal.y * 4095.;<br>  retVal.x = (retVal.x + 0.5) / 4096.;<br>  retVal.y = (retVal.y + 0.5) / 2.;<br>  return retVal;<br>}<br><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add LUT 1D processing for ocio_lut1d_0<br>  <br>  {<br>    outColor.r = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.r)).r;<br>    outColor.g = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.g)).r;<br>    outColor.b = texture(ocio_lut1d_0Sampler, ocio_lut1d_0_computePos(outColor.b)).r;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.9237070997046666, -0.20504593605401519, -0.023285583594355667, 0., -0.79501788145104413, 1.499361314350298, -0.42677817014753039, 0., -0.12868921825362739, -0.29431537829627863, 1.4500637537418828, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| D-Log D-Gamut | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.138583958, 0.138583958, 0.138583958);<br>    vec3 linear_segment_offset = vec3(0.0929045379, 0.0929045379, 0.0929045379);<br>    vec3 linear_segment_slopeinv = vec3(0.165956274, 0.165956274, 0.165956274);<br>    vec3 lin_slopeinv = vec3(1.0109179, 1.0109179, 1.0109179);<br>    vec3 lin_offset = vec3(0.010800000000000001, 0.010800000000000001, 0.010800000000000001);<br>    vec3 log_slopeinv = vec3(3.89616013, 3.89616013, 3.89616013);<br>    vec3 log_base = vec3(10., 10., 10.);<br>    vec3 log_offset = vec3(0.58455504907396005, 0.58455504907396005, 0.58455504907396005);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6747885840530357, -0.098555271555729629, -0.040971740853192197, 0., -0.57671111023511967, 1.3368391163574227, -0.24845494074321658, 0., -0.098077473817920285, -0.23828384480168621, 1.2894266815964019, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear D-Gamut | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6747885840530357, -0.098555271555729629, -0.040971740853192197, 0., -0.57671111023511967, 1.3368391163574227, -0.24845494074321658, 0., -0.098077473817920285, -0.23828384480168621, 1.2894266815964019, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| V-Log V-Gamut | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.180999666, 0.180999666, 0.180999666);<br>    vec3 linear_segment_offset = vec3(0.12499956, 0.12499956, 0.12499956);<br>    vec3 linear_segment_slopeinv = vec3(0.17857109, 0.17857109, 0.17857109);<br>    vec3 lin_slopeinv = vec3(1., 1., 1.);<br>    vec3 lin_offset = vec3(0.0087299999999999999, 0.0087299999999999999, 0.0087299999999999999);<br>    vec3 log_slopeinv = vec3(4.1405468, 4.1405468, 4.1405468);<br>    vec3 log_base = vec3(10., 10., 10.);<br>    vec3 log_offset = vec3(0.59820600000000002, 0.59820600000000002, 0.59820600000000002);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.8065758837249319, -0.17009034310239998, -0.025205784016640309, 0., -0.69569727408369764, 1.3059552160953665, -0.154468329360574, 0., -0.11087860964123716, -0.13586487299296365, 1.1796741133772104, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear V-Gamut | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.8065758837249319, -0.17009034310239998, -0.025205784016640309, 0., -0.69569727408369764, 1.3059552160953665, -0.154468329360574, 0., -0.11087860964123716, -0.13586487299296365, 1.1796741133772104, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Log3G10 REDWideGamutRGB | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0., 0., 0.);<br>    vec3 linear_segment_offset = vec3(0.15192689, 0.15192689, 0.15192689);<br>    vec3 linear_segment_slopeinv = vec3(0.0658211336, 0.0658211336, 0.0658211336);<br>    vec3 lin_slopeinv = vec3(0.00641127024, 0.00641127024, 0.00641127024);<br>    vec3 lin_offset = vec3(2.5597532699999999, 2.5597532699999999, 2.5597532699999999);<br>    vec3 log_slopeinv = vec3(4.45867252, 4.45867252, 4.45867252);<br>    vec3 log_base = vec3(10., 10., 10.);<br>    vec3 log_offset = vec3(0., 0., 0.);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.9819760179564536, -0.17814318205504703, -0.10179596596933896, 0., -0.90043183664045945, 1.5004683579162101, -0.53526345921182183, 0., -0.081544181316000275, -0.32232517586116, 1.6370594251811619, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear REDWideGamutRGB | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.9819760179564536, -0.17814318205504703, -0.10179596596933896, 0., -0.90043183664045945, 1.5004683579162101, -0.53526345921182183, 0., -0.081544181316000275, -0.32232517586116, 1.6370594251811619, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| S-Log3 S-Gamut3 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.167360976, 0.167360976, 0.167360976);<br>    vec3 linear_segment_offset = vec3(0.0928641111, 0.0928641111, 0.0928641111);<br>    vec3 linear_segment_slopeinv = vec3(0.151013061, 0.151013061, 0.151013061);<br>    vec3 lin_slopeinv = vec3(0.189999998, 0.189999998, 0.189999998);<br>    vec3 lin_offset = vec3(0.052631578947368397, 0.052631578947368397, 0.052631578947368397);<br>    vec3 log_slopeinv = vec3(3.91204596, 3.91204596, 3.91204596);<br>    vec3 log_base = vec3(10., 10., 10.);<br>    vec3 log_offset = vec3(0.410557184750733, 0.410557184750733, 0.410557184750733);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.8778156775191697, -0.17747979963189922, -0.02590143482982572, 0., -0.79127608338063604, 1.353784194250401, -0.15358565720579506, 0., -0.086539594138541606, -0.17630439461849309, 1.1794870920356171, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| S-Log3 S-Gamut3.Cine | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.167360976, 0.167360976, 0.167360976);<br>    vec3 linear_segment_offset = vec3(0.0928641111, 0.0928641111, 0.0928641111);<br>    vec3 linear_segment_slopeinv = vec3(0.151013061, 0.151013061, 0.151013061);<br>    vec3 lin_slopeinv = vec3(0.189999998, 0.189999998, 0.189999998);<br>    vec3 lin_offset = vec3(0.052631578947368397, 0.052631578947368397, 0.052631578947368397);<br>    vec3 log_slopeinv = vec3(3.91204596, 3.91204596, 3.91204596);<br>    vec3 log_base = vec3(10., 10., 10.);<br>    vec3 log_offset = vec3(0.410557184750733, 0.410557184750733, 0.410557184750733);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6268564003175596, -0.17910943044878672, -0.044166477894998633, 0., -0.53698863655119955, 1.4208630414432488, -0.20151920061693746, 0., -0.089867763766365294, -0.24175361099445788, 1.2456856785119332, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| S-Log3 Venice S-Gamut3 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.167360976, 0.167360976, 0.167360976);<br>    vec3 linear_segment_offset = vec3(0.0928641111, 0.0928641111, 0.0928641111);<br>    vec3 linear_segment_slopeinv = vec3(0.151013061, 0.151013061, 0.151013061);<br>    vec3 lin_slopeinv = vec3(0.189999998, 0.189999998, 0.189999998);<br>    vec3 lin_offset = vec3(0.052631578947368397, 0.052631578947368397, 0.052631578947368397);<br>    vec3 log_slopeinv = vec3(3.91204596, 3.91204596, 3.91204596);<br>    vec3 log_base = vec3(10., 10., 10.);<br>    vec3 log_offset = vec3(0.410557184750733, 0.410557184750733, 0.410557184750733);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.9901688195570184, -0.19613579460920139, -0.036624086206908393, 0., -0.9515515415040825, 1.3917665273557032, -0.14443018284830009, 0., -0.038617278052943085, -0.19563073274649953, 1.1810542690552108, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| S-Log3 Venice S-Gamut3.Cine | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Log 'Camera Log to Lin' processing<br>  <br>  {<br>    vec3 log_break = vec3(0.167360976, 0.167360976, 0.167360976);<br>    vec3 linear_segment_offset = vec3(0.0928641111, 0.0928641111, 0.0928641111);<br>    vec3 linear_segment_slopeinv = vec3(0.151013061, 0.151013061, 0.151013061);<br>    vec3 lin_slopeinv = vec3(0.189999998, 0.189999998, 0.189999998);<br>    vec3 lin_offset = vec3(0.052631578947368397, 0.052631578947368397, 0.052631578947368397);<br>    vec3 log_slopeinv = vec3(3.91204596, 3.91204596, 3.91204596);<br>    vec3 log_base = vec3(10., 10., 10.);<br>    vec3 log_offset = vec3(0.410557184750733, 0.410557184750733, 0.410557184750733);<br>    vec3 isAboveBreak = vec3(greaterThan( outColor.rgb, log_break));<br>    vec3 linSeg = ( outColor.rgb - linear_segment_offset ) * linear_segment_slopeinv;<br>    vec3 logSeg = (outColor.rgb - log_offset) * log_slopeinv;<br>    logSeg = pow(log_base, logSeg);<br>    logSeg = lin_slopeinv * (logSeg - lin_offset);<br>    outColor.rgb = isAboveBreak * logSeg + ( vec3(1., 1., 1.) - isAboveBreak ) * linSeg;<br>  }<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.725635767457687, -0.19552629878430919, -0.053585711071423188, 0., -0.69113740569172921, 1.4589135591606734, -0.19353475807298107, 0., -0.034498361765968752, -0.26338726037635624, 1.247120469144404, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear S-Gamut3 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.8778156775191697, -0.17747979963189922, -0.02590143482982572, 0., -0.79127608338063604, 1.353784194250401, -0.15358565720579506, 0., -0.086539594138541606, -0.17630439461849309, 1.1794870920356171, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear S-Gamut3.Cine | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.6268564003175596, -0.17910943044878672, -0.044166477894998633, 0., -0.53698863655119955, 1.4208630414432488, -0.20151920061693746, 0., -0.089867763766365294, -0.24175361099445788, 1.2456856785119332, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear Venice S-Gamut3 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.9901688195570184, -0.19613579460920139, -0.036624086206908393, 0., -0.9515515415040825, 1.3917665273557032, -0.14443018284830009, 0., -0.038617278052943085, -0.19563073274649953, 1.1810542690552108, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Linear Venice S-Gamut3.Cine | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Matrix processing<br>  <br>  {<br>    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);<br>    vec4 tmp = res;<br>    res = mat4(1.725635767457687, -0.19552629878430919, -0.053585711071423188, 0., -0.69113740569172921, 1.4589135591606734, -0.19353475807298107, 0., -0.034498361765968752, -0.26338726037635624, 1.247120469144404, 0., 0., 0., 0., 1.) * tmp;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
| Camera Rec.709 | lin_rec709 | <code><br>// Declaration of the OCIO shader function<br><br>vec4 OCIOMain(vec4 inPixel)<br>{<br>  vec4 outColor = inPixel;<br>  <br>  // Add Gamma 'monCurveFwd' processing<br>  <br>  {<br>    vec4 breakPnt = vec4(0.0810000002, 0.0810000002, 0.0810000002, 1.);<br>    vec4 slope = vec4(0.221543506, 0.221543506, 0.221543506, 1.);<br>    vec4 scale = vec4(0.909918129, 0.909918129, 0.909918129, 0.999998987);<br>    vec4 offset = vec4(0.0900818929, 0.0900818929, 0.0900818929, 9.99998974e-07);<br>    vec4 gamma = vec4(2.22222233, 2.22222233, 2.22222233, 1.00000095);<br>    vec4 isAboveBreak = vec4(greaterThan( outColor, breakPnt));<br>    vec4 linSeg = outColor * slope;<br>    vec4 powSeg = pow( max( vec4(0., 0., 0., 0.), scale * outColor + offset), gamma);<br>    vec4 res = isAboveBreak * powSeg + ( vec4(1., 1., 1., 1.) - isAboveBreak ) * linSeg;<br>    outColor.rgb = vec3(res.x, res.y, res.z);<br>    outColor.a = res.w;<br>  }<br><br>  return outColor;<br>}<br></code>|
</details>

### Integrating OCIO with MaterialX

We will pick an example transform to go over details on mapping from OCIO to MaterialX.

The first thing of note is OCIO function signatures

* Currently all signatures transform 4 channel color inputs while MaterialX supports both 3 and 4 channel variants. This can be easily handled by adding pre and post conversion nodes, or by creating variant function signatures. The former is more robust and more in line with the proposed direction to have all OCIO transforms to be represented as graphs.

* The signature name is not unique. This can be handled as OCIO provides mechanism to override the function names using `setFunctionName` and `setResourcePrefix`.

Following the current MaterialX convention we use the signature notation:
`mx_<sourceName>_to_<targetname>_<type>` where `type` is either `color3` or `color4` for 3 or 4 channel variants.

We add in two new utilities:

1. `createTransformName` which will generate the unique function name
2. `setShaderDescriptionParameters` which overrides the function name but also adds a prefix to uniquely identify dependent resources.

These are then used in a new code generation variation called `generateShaderCode2()` which has additionally been modified to return the number of dependent texture resources, which can be queried from the shader 
descriptor via the `GpuShaderDesc.getTextures()` iterator.

In [8]:
def createTransformName(sourceSpace, targetSpace, typeName): 
    transformFunctionName = "mx_" + mx.createValidName(sourceSpace) + "_to_" + targetSpace + "_" + typeName 
    return transformFunctionName

def setShaderDescriptionParameters(shaderDesc, sourceSpace, targetSpace, typeName):
    transformFunctionName = createTransformName(sourceSpace, targetSpace, typeName)
    shaderDesc.setFunctionName(transformFunctionName)
    shaderDesc.setResourcePrefix(transformFunctionName)

def generateShaderCode2(config, sourceColorSpace, destColorSpace, language):
    shaderCode = ''
    textureCount = 0
    if not config:
        return shaderCode, textureCount

    # Create a processor for a pair of colorspaces (namely to go to linear)
    processor = None
    try:
        processor = config.getProcessor(sourceColorSpace, destColorSpace)
    except:
        print('Failed to generated code for transform: %s -> %s' % (sourceColorSpace, destColorSpace))
        return shaderCode, textureCount

    if processor:
        processor = processor.getOptimizedProcessor(OCIO.OPTIMIZATION_ALL)
        gpuProcessor = processor.getDefaultGPUProcessor()
        if gpuProcessor:
            shaderDesc = OCIO.GpuShaderDesc.CreateShaderDesc()
            if shaderDesc:
                try:
                    shaderDesc.setLanguage(language)
                    if shaderDesc.getLanguage() == language:
                        setShaderDescriptionParameters(shaderDesc, sourceColorSpace, destColorSpace, "color4")
                        gpuProcessor.extractGpuShaderInfo(shaderDesc)                                                                 
                        shaderCode = shaderDesc.getShaderText()
                        for t in shaderDesc.getTextures():
                            textureCount += 1
                except OCIO.Exception as err:
                    print(err)
    
    return shaderCode, textureCount


### OCIO Resource Dependencies

Resource dependencies is a second major issue to examine.

In the example below we convert two different source color spaces.  

One is "self-contained" in that there are no support functions being produced (`ACES2065-1`),
while the second adds additional function and resources. Note that we maintain uniqueness of these additions by using `setFunctionName` and `setResourcePrefix` respectively.

#### Example 1: Self-contained

In [9]:
sourceColorSpace = 'ACES2065-1' # "acescg"
textureCount = 0
code = ''
code, textureCount = generateShaderCode2(builtinCfgC, sourceColorSpace, targetColorSpace, language)
if code:
    code = code.replace("// Declaration of the OCIO shader function\n", 
                        "// " + sourceColorSpace + " to " + targetColorSpace + " function. Texture count: %d\n" % textureCount)
    code = '```c++\n' + code + '\n```'
    display_markdown(code, raw=True)    

```c++

// ACES2065-1 to lin_rec709 function. Texture count: 0

vec4 mx_ACES2065_1_to_lin_rec709_color4(vec4 inPixel)
{
  vec4 outColor = inPixel;
  
  // Add Matrix processing
  
  {
    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);
    vec4 tmp = res;
    res = mat4(2.5216861867438798, -0.27647991422992202, -0.015378064966034201, 0., -1.1341309882397199, 1.37271908766826, -0.152975335867399, 0., -0.38755519850416398, -0.096239173438334005, 1.16835340083343, 0., 0., 0., 0., 1.) * tmp;
    outColor.rgb = vec3(res.x, res.y, res.z);
    outColor.a = res.w;
  }

  return outColor;
}

```

#### Example 2: Secondary Dependencies 

In [10]:
sourceColorSpace = 'ACEScc' # "acescg"
code, textureCount = generateShaderCode2(builtinCfgC, sourceColorSpace, targetColorSpace, language)
if code:
    code = code.replace("// Declaration of the OCIO shader function\n", 
                        "// " + sourceColorSpace + " to " + targetColorSpace + " function. Texture count: %d\n" % textureCount)
    code = '```c++\n' + code + '\n```\n'

    md = '<details><summary>Secondary Dependency Sample Code</summary>\n\n' + code + '</details>'
    display_markdown(md, raw=True)

<details><summary>Secondary Dependency Sample Code</summary>

```c++

// Declaration of all textures

uniform sampler2D mx_ACEScc_to_lin_rec709_color4_lut1d_0Sampler;

// Declaration of all helper methods

vec2 mx_ACEScc_to_lin_rec709_color4_lut1d_0_computePos(float f)
{
  float dep = clamp(f, 0.0, 1.0) * 4095.;
  vec2 retVal;
  retVal.y = floor(dep / 4095.);
  retVal.x = dep - retVal.y * 4095.;
  retVal.x = (retVal.x + 0.5) / 4096.;
  retVal.y = (retVal.y + 0.5) / 2.;
  return retVal;
}

// ACEScc to lin_rec709 function. Texture count: 1

vec4 mx_ACEScc_to_lin_rec709_color4(vec4 inPixel)
{
  vec4 outColor = inPixel;
  
  // Add Range processing
  
  {
    outColor.rgb = outColor.rgb * vec3(0.53763440860215062, 0.53763440860215062, 0.53763440860215062) + vec3(0.19354838709677422, 0.19354838709677422, 0.19354838709677422);
    outColor.rgb = max(vec3(0., 0., 0.), outColor.rgb);
    outColor.rgb = min(vec3(1., 1., 1.), outColor.rgb);
  }
  
  // Add LUT 1D processing for mx_ACEScc_to_lin_rec709_color4_lut1d_0
  
  {
    outColor.r = texture(mx_ACEScc_to_lin_rec709_color4_lut1d_0Sampler, mx_ACEScc_to_lin_rec709_color4_lut1d_0_computePos(outColor.r)).r;
    outColor.g = texture(mx_ACEScc_to_lin_rec709_color4_lut1d_0Sampler, mx_ACEScc_to_lin_rec709_color4_lut1d_0_computePos(outColor.g)).r;
    outColor.b = texture(mx_ACEScc_to_lin_rec709_color4_lut1d_0Sampler, mx_ACEScc_to_lin_rec709_color4_lut1d_0_computePos(outColor.b)).r;
  }
  
  // Add Matrix processing
  
  {
    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);
    vec4 tmp = res;
    res = mat4(0.69545224135745187, 0.044794563372037723, -0.0055258825581135443, 0., 0.14067869647029416, 0.85967111845642175, 0.0040252103059786595, 0., 0.16386906217225405, 0.0955343181715404, 1.0015006722521349, 0., 0., 0., 0., 1.) * tmp;
    outColor.rgb = vec3(res.x, res.y, res.z);
    outColor.a = res.w;
  }
  
  // Add Range processing
  
  {
    outColor.rgb = max(vec3(0., 0., 0.), outColor.rgb);
  }
  
  // Add Matrix processing
  
  {
    vec4 res = vec4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);
    vec4 tmp = res;
    res = mat4(2.5216861867438798, -0.27647991422992202, -0.015378064966034201, 0., -1.1341309882397199, 1.37271908766826, -0.152975335867399, 0., -0.38755519850416398, -0.096239173438334005, 1.16835340083343, 0., 0., 0., 0., 1.) * tmp;
    outColor.rgb = vec3(res.x, res.y, res.z);
    outColor.a = res.w;
  }

  return outColor;
}

```
</details>

#### Issues With Texture Resources

From an integration point of view any introduction of texture lookups requires resource declarations in the code. (such as the `uniform sampler2D mx_ACEScc_to_lin_rec709_color4_ocio_lut1d_0Sampler;` sampler declaration).

1. The only way to handle these is to have additional logic added for code insertion of color transforms, such that the shader function declarations and resources can be inserted into the code independently. The current MaterialX code generation logic does not otherwise support this using the "default color system".

    > Note : An experiment was attempted previously but does not align with the current proposal to have stand-alone node definitions. It was thus abandoned. ( For those interested the full code with code changes can be found <a href="https://github.com/autodesk-forks/MaterialX/pull/1379/files#diff-a181860f6edce31fab2f260d982a9363ef80062b7eea99f63a19cf0ea60ee44f">here</a>). Here 1D lookups (LUTS) were specified as input arrays and code generation created 1D textures dynamically based on the array inputs. 3D lookups were not handled.

2. **From the point of view of creating node graphs, any implementation resource dependencies means it cannot be cleanly wrapped up into a self-contained node definition and implementation.**

For now these can be "skipped" until such time as they are require, or the implementation changes
to avoid using these.

#### Finding Transforms Using Texture Resources

We can re-iterate through all of the transforms of interest, and find these transforms using
the following code. 
> Note that the code generation is not necessary but is written this way to 
reuse the existing utility `generateShaderCode2`).

In [11]:
# Scan through all the color spaces on the configs to check for texture resource usage.
testedSources = set()
for c in configs:
    config = OCIO.Config.CreateFromBuiltinConfig(c)
    colorSpaces = config.getColorSpaces()
    for colorSpace in colorSpaces:
        colorSpaceName = colorSpace.getName()
        # Skip if the colorspace is already tested
        if colorSpaceName in testedSources:
            continue
        testedSources.add(colorSpaceName)

        # Test for texture resource usage
        code, textureCount = generateShaderCode2(config, colorSpace.getName(), targetColorSpace, language)
        if textureCount:
            print('- Transform "%s" to "%s" requires %d texture resources' % (colorSpace.getName(), targetColorSpace, textureCount))

- Transform "Rec.2100-PQ - Display" to "lin_rec709" requires 1 texture resources
- Transform "ST2084-P3-D65 - Display" to "lin_rec709" requires 1 texture resources
- Transform "ACEScc" to "lin_rec709" requires 1 texture resources
- Transform "Rec.2100-HLG - Display" to "lin_rec709" requires 1 texture resources
- Transform "ADX10" to "lin_rec709" requires 1 texture resources


- Transform "ADX16" to "lin_rec709" requires 1 texture resources
- Transform "Apple Log" to "lin_rec709" requires 1 texture resources
- Transform "CanonLog2 CinemaGamut D55" to "lin_rec709" requires 1 texture resources
- Transform "CanonLog3 CinemaGamut D55" to "lin_rec709" requires 1 texture resources


### Target Language Support

At time of writing the target languages supported by OCIO and MaterialX differ. This includes non-core support such as `MDL` and current versions of `OSL`. Also as no logical operators are provided as with MaterialX,  targets such as `Vex` which parses and maps MaterialX nodes as operators is not easy to do.

OCIO and MaterialX recently added in `Metal` language support. It is to be checked if there would be any issues with the additional `struct` wrappers required for this language as it is uncommon for MaterialX code generation to call into a `struct` function at the current time.

In [12]:
sourceColorSpace = "acescg"
language = OCIO.GpuLanguage.GPU_LANGUAGE_MSL_2_0
code, textureCount = generateShaderCode2(builtinCfgC, sourceColorSpace, targetColorSpace, language)
if code:
    code = code.replace("// Declaration of the OCIO shader function\n", "// " + sourceColorSpace + " to " + targetColorSpace + " function\n")
    code = '```c++\n' + code + '\n```\n'
    md = '<details><summary>MSL struct usage</summary>\n\n' + code + '</details>'
    display_markdown(md, raw=True)

<details><summary>MSL struct usage</summary>

```c++

// Declaration of class wrapper

struct mx_acescg_to_lin_rec709_color4mx_acescg_to_lin_rec709_color4
{
mx_acescg_to_lin_rec709_color4mx_acescg_to_lin_rec709_color4(
)
{
}



// acescg to lin_rec709 function

float4 mx_acescg_to_lin_rec709_color4(float4 inPixel)
{
  float4 outColor = inPixel;
  
  // Add Matrix processing
  
  {
    float4 res = float4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);
    float4 tmp = res;
    res = float4x4(1.7050509926579815, -0.1302564175070435, -0.024003356804618046, 0., -0.62179212065700573, 1.140804736575405, -0.12896897606497093, 0., -0.083258872000979672, -0.010548319068357662, 1.1529723328695858, 0., 0., 0., 0., 1.) * tmp;
    outColor.rgb = float3(res.x, res.y, res.z);
    outColor.a = res.w;
  }

  return outColor;
}

// Close class wrapper


};
float4 mx_acescg_to_lin_rec709_color4(
  float4 inPixel)
{
  return mx_acescg_to_lin_rec709_color4mx_acescg_to_lin_rec709_color4(
  ).mx_acescg_to_lin_rec709_color4(inPixel);
}

```
</details>

Using version 2.3 to access `OSL` there appears to be additional issues with the code
generated as additional utility functions may be inserted which are not renamed to avoid collisions.

For example functions called `max()`, `pow()` etc are added which are outside the scope of the main shader declaration for which do not seem to be included in the logic for unique function name. As well as include additional include files which should be part of the OSL distribution. These need to be stripped out to embed this code as part of a larger OSL shader which already includes these functions to avoid name clashes.
An <a href="https://github.com/AcademySoftwareFoundation/OpenColorIO/issues/1851" target="_blank">issue</a> has been logged for this.

As `OSL` integrations will generally perform color management outside of the shader, it is to be seen if this is important enough to address.

In [13]:
if OCIO.GpuLanguage.LANGUAGE_OSL_1:
    sourceColorSpace = "acescg"
    language = OCIO.GpuLanguage.LANGUAGE_OSL_1
    code, textureCount = generateShaderCode2(builtinCfgC, sourceColorSpace, targetColorSpace, language)
    if code:
        # Bit of ugly patching to make the main function name consistent.
        transformName = createTransformName(sourceColorSpace, targetColorSpace, 'color4')
        code = code.replace('OSL_' + transformName, '__temp_name__')
        code = code.replace(transformName, transformName + '_impl')
        code = code.replace('__temp_name__', transformName)
        code = code.replace("// Declaration of the OCIO shader function\n", "// " + sourceColorSpace + " to " + targetColorSpace + " function\n")
        code = '```c++\n' + code + '\n```\n'
        md = '<details><summary>OSL dependent function / includes code</summary>\n\n' + code + '</details>'
        display_markdown(md, raw=True)

<details><summary>OSL dependent function / includes code</summary>

```c++

/* All the includes */

#include "vector4.h"
#include "color4.h"

/* All the generic helper methods */

vector4 __operator__mul__(matrix m, vector4 v)
{
  return transform(m, v);
}

vector4 __operator__mul__(color4 c, vector4 v)
{
  return vector4(c.rgb.r, c.rgb.g, c.rgb.b, c.a) * v;
}

vector4 __operator__mul__(vector4 v, color4 c)
{
  return v * vector4(c.rgb.r, c.rgb.g, c.rgb.b, c.a);
}

vector4 __operator__sub__(color4 c, vector4 v)
{
  return vector4(c.rgb.r, c.rgb.g, c.rgb.b, c.a) - v;
}

vector4 __operator__add__(vector4 v, color4 c)
{
  return v + vector4(c.rgb.r, c.rgb.g, c.rgb.b, c.a);
}

vector4 __operator__add__(color4 c, vector4 v)
{
  return vector4(c.rgb.r, c.rgb.g, c.rgb.b, c.a) + v;
}

vector4 pow(color4 c, vector4 v)
{
  return pow(vector4(c.rgb.r, c.rgb.g, c.rgb.b, c.a), v);
}

vector4 max(vector4 v, color4 c)
{
  return max(v, vector4(c.rgb.r, c.rgb.g, c.rgb.b, c.a));
}

/* The shader implementation */

shader mx_acescg_to_lin_rec709_color4(color4 inColor = {color(0), 1}, output color4 outColor = {color(0), 1})
{


// acescg to lin_rec709 function

color4 mx_acescg_to_lin_rec709_color4_impl(color4 inPixel)
{
  color4 outColor = inPixel;
  
  // Add Matrix processing
  
  {
    vector4 res = vector4(outColor.rgb.r, outColor.rgb.g, outColor.rgb.b, outColor.a);
    vector4 tmp = res;
    res = matrix(1.7050509926579815, -0.1302564175070435, -0.024003356804618046, 0., -0.62179212065700573, 1.140804736575405, -0.12896897606497093, 0., -0.083258872000979672, -0.010548319068357662, 1.1529723328695858, 0., 0., 0., 0., 1.) * tmp;
    outColor.rgb = vector(res.x, res.y, res.z);
    outColor.a = res.w;
  }

  return outColor;
}

outColor = mx_acescg_to_lin_rec709_color4_impl(inColor);
}

```
</details>

### Creating MaterialX Node Definitions / Implementation

Given source code for now, it is still possible to create implementations and node definitions. If in the future
the implementations can become MaterialX node graphs then the definition interface can still be used. 

To create a new definition:
1. The source and target color space is used to derive: 
   * a transform name: using the previously described `createTransformName()` utility
   * a node name by replace the 'mx_' function name with the "standard" 'ND_' prefix used for definition
   * a node category 
2. Use `addNodeDef()` API to add a new definition
3. Set node group to be `colortransform` which is the recommended group for new color transforms.
4. Add a single input and output of the appropriate type (`color3` or `color4`).
5. Set a default value on the input. For this code we assume the defaul to be opaque black. 

Thie logic is encapsulated in a new `generateMaterialXDefinition()` utility function.

In [14]:
def generateMaterialXDefinition(doc, sourceColorSpace, targetColorSpace, inputName, type):

    # Create a definition
    transformName = createTransformName(sourceColorSpace, targetColorSpace, type)
    nodeName = transformName.replace('mx_', 'ND_')

    comment = doc.addChildOfCategory('comment')
    comment.setDocString(' Color space %s to %s transform. Generated via OCIO. ' % (sourceColorSpace, targetColorSpace))

    definition = doc.addNodeDef(nodeName, 'color4')
    category = sourceColorSpace + '_to_' + targetColorSpace
    definition.setNodeString(category)
    definition.setNodeGroup('colortransform')

    defaultValueString = '0.0 0.0 0.0 1.0'
    defaultValue = mx.createValueFromStrings(defaultValueString, 'color4')
    input = definition.addInput(inputName, type)
    input.setValue(defaultValue)
    output = definition.getOutput('out')
    output.setAttribute('default', 'in')

    return definition


Another utility called `writeShaderCode()` is used to write the code to file following the "standard" MaterialX naming convention.

In [15]:

def writeShaderCode(code, transformName, extension, target):

    # Write source code file
    filename = mx.FilePath('./data') / mx.FilePath(transformName + '.' + extension)
    print('Write target[%s] source file %s' % (target,filename.asString()))
    f = open(filename.asString(), 'w')
    f.write(code)
    f.close()


The implementation creation logic is encapsulated in a `createMaterialXImplementation()` utility function which appends a new implementation to an existing Document.

The transform name is assumged to be precreated using `createTransformName()`.

This is used to create a unique implementation Element name, and source code filename. We decied to use a file on disk as it is not possible to inline 1 or more function functions created by OCIO.

The implementation is associated with an existing node definition which is passed in and a target is explicit set to indicate which shading language (target) the implementation is for. 

In [16]:

def createMaterialXImplementation(doc, definition, transformName, extension, target):
    '''
    Create a new implementation in a document for a given definition.
    '''
    implName = transformName + '_' + target
    filename = transformName + '.' + extension
    implName = implName.replace('mx_', 'IM_')

    # Check if implementation already exists
    impl = doc.getImplementation(implName)
    if impl:
        print('Implementation already exists: %s' % implName)
        return impl

    comment = doc.addChildOfCategory('comment')
    comment.setDocString(' Color space %s to %s transform. Generated via OCIO for target: %s' 
                         % (sourceColorSpace, targetColorSpace, target))
    impl = doc.addImplementation(implName)
    impl.setFile(filename)
    impl.setFunction(transformName)
    impl.setTarget(target)
    impl.setNodeDef(definition)

    return impl

Finally a small utility is added to write the document to disk.

In [17]:
def writeDocument(doc, filename):
    print('Write MaterialX file:', filename.asString())
    mx.writeToXmlFile(doc, filename)

Using these utilities we: 
- Create separate definition and implementation Documents. 
- Generate shader code for `GLSL`, `MSL`, and `OSL`` for the same color transform. (`OSL` is available in version 2.3).
- Create a new definition for that transform
- Create a new implementation for each shader code result

In [18]:
# Example to create:
# - source code for a given transform for 2 shader targets
# - A definition wrapper for the source
# - An implementations per source code. All implementations are associated with single definition
definitionDoc = mx.createDocument()
definition = None

implDoc = mx.createDocument()

sourceColorSpace = "acescg"
type = 'color4'
transformName = createTransformName(sourceColorSpace, targetColorSpace, type)

# All code has the same input name
# It is possible to use a different name than the name used in the generated function ('inPixel')
#IN_PIXEL_STRING = 'inPixel'
IN_PIXEL_STRING = 'in'

# Pick a source and target color space
sourceColorSpace = 'acescg'
targetColorSpace = 'lin_rec709'

# List of MaterialX target language, source code extensions, and OCIO GPU languages
generationList = [#['genmsl', 'metal', OCIO.GpuLanguage.GPU_LANGUAGE_MSL_2_0],
                  ['genglsl', 'glsl', OCIO.GpuLanguage.GPU_LANGUAGE_GLSL_4_0] ]

if OCIO.GpuLanguage.LANGUAGE_OSL_1:
    generationList.append(['genosl', 'osl', OCIO.GpuLanguage.LANGUAGE_OSL_1])

for gen in generationList:
    target = gen[0]
    extension = gen[1]
    language = gen[2]

    code, textureCount = generateShaderCode2(builtinCfgC, sourceColorSpace, targetColorSpace, language)
    if code:
        # Emit the source code file
        writeShaderCode(code, transformName, extension, target)

        # Create the definition once
        if not definition:
            definition = generateMaterialXDefinition(definitionDoc, sourceColorSpace, targetColorSpace, 
                                                    IN_PIXEL_STRING, type)
        
        # Create the implementation
        createMaterialXImplementation(implDoc, definition, transformName, extension, target)

Write target[genglsl] source file ./data/mx_acescg_to_lin_rec709_color4.glsl
Write target[genosl] source file ./data/mx_acescg_to_lin_rec709_color4.osl


The resulting MaterialX wrappers are then written to disk as follows:

### Variant Creation

Given a node definition for the `color4` variant is is possible to create a functional graph and corresponding definition for a `color3` variant. The graph simple convertes from `color3` to `color4` before connecting to the transform and then converts back to `color3` for output.

In [19]:
color4Name = definition.getName()
color3Name = color4Name.replace('color4', 'color3')
color3Def = definitionDoc.addNodeDef(color3Name)
color3Def.copyContentFrom(definition)
c3input = color3Def.getInput(IN_PIXEL_STRING)
c3input.setType('color3')
c3input.setValue(mx.createValueFromStrings('0.0 0.0 0.0', 'color3'))
    
ngName = color3Def.getName().replace('ND_', 'NG_')
ng = definitionDoc.addNodeGraph(ngName)
c4instance = ng.addNodeInstance(definition)
c4instance.addInputsFromNodeDef()
c4instanceIn = c4instance.getInput(IN_PIXEL_STRING)
c3to4 = ng.addNode('convert', 'c3to4', 'color4')
c3to4Input = c3to4.addInput('in', 'color3')
c4to3 = ng.addNode('convert', 'c4to3', 'color3')
c4to3Input = c4to3.addInput('in', 'color4')
ngout = ng.addOutput('out', 'color3')
#ngin = ng.addInput('in', 'color3')
ng.setNodeDef(color3Def)

c4instanceIn.setNodeName(c3to4.getName())
c4to3Input.setNodeName(c4instance.getName())
ngout.setNodeName(c4to3.getName())
c3to4Input.setInterfaceName(IN_PIXEL_STRING)

result = mx.writeToXmlString(definitionDoc)
display_markdown('#### Generated Color3 Variant\n' + '```xml\n' + result + '```\n', raw=True)

valid, log = definitionDoc.validate()
if not valid:
    print('Document created is invalid:', log)

filename = mx.FilePath('./data') / mx.FilePath(definition.getName() + '_2.' + 'mtlx')
print('Write MaterialX definition variant file:', filename.asString())
mx.writeToXmlFile(definitionDoc, filename)

#### Generated Color3 Variant
```xml
<?xml version="1.0"?>
<materialx version="1.39">
  <!-- Color space acescg to lin_rec709 transform. Generated via OCIO. -->
  <nodedef name="ND_acescg_to_lin_rec709_color4" node="acescg_to_lin_rec709" nodegroup="colortransform">
    <output name="out" type="color4" default="in" />
    <input name="in" type="color4" value="0, 0, 0, 1" />
  </nodedef>
  <nodedef name="ND_acescg_to_lin_rec709_color3" node="acescg_to_lin_rec709" nodegroup="colortransform">
    <output name="out" type="color3" />
    <input name="in" type="color3" value="0, 0, 0" />
  </nodedef>
  <nodegraph name="NG_acescg_to_lin_rec709_color3" nodedef="ND_acescg_to_lin_rec709_color3">
    <acescg_to_lin_rec709 name="node1" type="color4" nodedef="ND_acescg_to_lin_rec709_color4">
      <input name="in" type="color4" value="0, 0, 0, 1" nodename="c3to4" />
    </acescg_to_lin_rec709>
    <convert name="c3to4" type="color4">
      <input name="in" type="color3" interfacename="in" />
    </convert>
    <convert name="c4to3" type="color3">
      <input name="in" type="color4" nodename="node1" />
    </convert>
    <output name="out" type="color3" nodename="c4to3" />
  </nodegraph>
</materialx>
```


Document created is invalid: Node input has too many bindings: <input name="in" type="color4" value="0, 0, 0, 1" nodename="c3to4">

Write MaterialX definition variant file: ./data/ND_acescg_to_lin_rec709_color4_2.mtlx


### MaterialX Standard Library Inclusion

It is possible to add a new color space transform to the "standard" MaterialX library locations. This can be done for local testing for in some cases when additional search paths are not supported. 

Introduced with version **1.38.7**, the location of definitions and implementations can be found in the `cmlib` folder under the installation `libraries` location. Note that target specific source file directories were removed as all implementations are now target independent node graphs.

To include source file implementations the appropriate `target` folders can be added in.
Under that folder an implementation file can be added for each target along with associated source files. The new definition can be added to `cmlib_defs.mtlx.

See the <a href="https://kwokcb.github.io/MaterialX_Learn/pymaterialx/mtlx_definitions_notebook.html" target="_blank">Creating Definitions</a> book for more on how set up new definitions.

The standard library folder structure is partially shown below. The bolded items would be the ones of interest. The `stdlib` folder is also shown to show how each folder follows the same naming and hierarchy conventions.

----
- cmlib // Color space transform library
    - **cmlib_defs.mtlx** // definitions file
    - cmlib_ng.mtlx       // functional node graphs file
    - **genglsl**         // GLSL implementation folder
        - **cmlib_genglsl_impl.mtlx** // Implementation declarations
        - **GLSL files reside here**        
    - **genmsl**          // MSL implementation folder
        - **cmlib_genmsl_impl.mtlx** // Implementation declarations
        - **MSL files reside here**        
- stdlib // "Standard" node library
    - genglsl
        - stdlib_genglsl_impl.mtlx
    - genmdl
    - genmsl
        - stdlib_genmsl_impl.mtlx
    - genosl
  - targets

----

### Future Exploration

At time of writing (September 2024), the NanoColor initiative is underway. The current plan is to allow pre-processing of transforms to create MaterialX definitions with graph implementation. See the next section for some unofficial prototyping.

As there is the intent to provide Javascript bindings it will be interesting to see how this will interact with Web libraries and how it can work with the glTF Texture Procedurals extension.

## NodeGraph Prototype

The following is some prototype code to extract out the list of transforms and 
attempt to create corresponding MaterialX nodes. It currently only handles matrix operations.

As a first step instead of extracting out code the list of transforms
is extracted. A GPU processor or CPU processor can be used. (Not sure if there is any difference which processor is used). This logic is given in the `generateTransformGraph` utility

In [20]:
def generateTransformGraph(config, sourceColorSpace, destColorSpace):
    if not config:
        return None

    # Create a processor for a pair of colorspaces (namely to go to linear)
    processor = None
    groupTransform = None
    try:
        processor = config.getProcessor(sourceColorSpace, destColorSpace)
    except:
        print('Failed to get processor for: %s -> %s' % (sourceColorSpace, destColorSpace))
        return groupTransform

    if processor:
        processor = processor.getOptimizedProcessor(OCIO.OPTIMIZATION_ALL) 
        groupTransform = processor.createGroupTransform()
    
    return groupTransform

Next we use this to extract out the list of transforms and iterate over them to pull out information per transform.

The basic 'template' for creating a new MaterialX definition is a single new `nodedef` with one input and one output. In this case we use `color3` as the input and output type.

For each transform a new corresponding MaterialX node is created. For now the logic only handles matrix tranforms. These are chained together incrementally. All computations work on `vector3` data.

All nodes are encapsulated into a `functional` nodegraph implementation with the appropriate conversion to/from `color3` to `vector3` is performed.

In [21]:

groupTransform = generateTransformGraph(builtinCfgC, sourceColorSpace, targetColorSpace)
result = f'{groupTransform}'
display_markdown('#### Extracted Transform\n' + '```xml\n' + result + '```\n', raw=True)

# To add. Proper testing of unsupported transforms...
invalidTransforms = [ OCIO.TransformType.TRANSFORM_TYPE_LUT3D, OCIO.TransformType.TRANSFORM_TYPE_LUT1D, 
                     OCIO.TransformType.TRANSFORM_TYPE_RANGE, 
                     OCIO.TransformType.TRANSFORM_TYPE_GRADING_PRIMARY ]

# Create a document, a nodedef and a functional graph.
graphDoc = mx.createDocument()
outputType = 'color3'
xformName = sourceColorSpace + '_to_' + targetColorSpace + '_' + outputType
nd = graphDoc.addNodeDef('ND_' + xformName )
nd.setAttribute('node', xformName)
ndInput = nd.addInput('in', 'color3')
ndInput.setValue(mx.createValueFromStrings('0.0 0.0 0.0', 'color3'))
ng = graphDoc.addNodeGraph('NG_' + xformName)
ng.setAttribute('nodedef', nd.getName())
convertNode = ng.addNode('convert', 'asVec', 'vector3')
converInput = convertNode.addInput('in', 'color3')
converInput.setInterfaceName('in')

print(f'Transform from: {sourceColorSpace} to {targetColorSpace}')
print(f'Number of transforms: {groupTransform.__len__()}')
previousNode = None

# Iterate and create appropriate nodes and connections
for i in range(groupTransform.__len__()):
    transform = groupTransform.__getitem__(i)
    # Get type of transform
    transformType = transform.getTransformType()
    if transformType in invalidTransforms:
        print(f'- Transform[{i}]: {transformType} contains an unsupported transform type')
        continue

    #print(f'- Transform[{i}]: {transformType}')   
    if transformType == OCIO.TransformType.TRANSFORM_TYPE_MATRIX:
        matrixNode = ng.addNode('transform', ng.createValidChildName(f'matrixTransform'), 'vector3')

        # Route output from previous node as input of current node
        inInput = matrixNode.addInput('in', 'vector3')
        if previousNode:            
            inInput.setAttribute('nodename', previousNode)
        else:
            if i==0:
                inInput.setAttribute('nodename', 'asVec')
            else:
                inInput.setValue(mx.createValueFromStrings('0.0 0.0 0.0', 'vector3'))

        # Set matrix value
        matInput = matrixNode.addInput('mat', 'matrix33')
        matrixValue = transform.getMatrix()
        # Extract 3x3 matrix from 4x4 matrix
        matrixValue = matrixValue[0:3] + matrixValue[4:7] + matrixValue[8:11]
        matrixValue = ', '.join([str(x) for x in matrixValue])
        #print('  - Matrix:', matrixValue)
        matInput.setAttribute('value', matrixValue)        

        # Add offset value
        offsetValue = transform.getOffset()
        offsetValue = ', '.join([str(x) for x in offsetValue])
        #print('  - Offset:', offsetValue)
        # Add a add vector3 to graph

        previousNode = matrixNode.getName()
    #elif transformType == OCIO.TransformType.TRANSFORM_TYPE_LOG:
    #    print('  - Base:', transform.getBase())

# Create an output for the last node if any
convertNode2 = ng.addNode('convert', 'asColor', 'color3')
converInput2 = convertNode2.addInput('in', 'vector3')
converInput2.setAttribute('nodename', previousNode)
if previousNode:
    out = ng.addOutput(ng.createValidChildName('out'), 'color3')
    out.setAttribute('nodename', 'asColor')

# Write the graph document
print('---------------------------')
print('Write OCIO transform graph file:', xformName + '.' + 'mtlx')
filename = mx.FilePath('./data') / mx.FilePath(xformName + '.' + 'mtlx')
mx.writeToXmlFile(graphDoc, filename)

result = mx.writeToXmlString(graphDoc)
display_markdown('#### Generated Transform Graph\n' + '```xml\n' + result + '```\n', raw=True)


#### Extracted Transform
```xml
<GroupTransform direction=forward, transforms=
        <MatrixTransform direction=forward, fileindepth=unknown, fileoutdepth=unknown, matrix=[1.705050992657982, -0.6217921206570057, -0.08325887200097967, 0, -0.1302564175070435, 1.140804736575405, -0.01054831906835766, 0, -0.02400335680461805, -0.1289689760649709, 1.152972332869586, 0, 0, 0, 0, 1], offset=[0, 0, 0, 0]>>```


Transform from: acescg to lin_rec709
Number of transforms: 1
---------------------------
Write OCIO transform graph file: acescg_to_lin_rec709_color3.mtlx


#### Generated Transform Graph
```xml
<?xml version="1.0"?>
<materialx version="1.39">
  <nodedef name="ND_acescg_to_lin_rec709_color3" node="acescg_to_lin_rec709_color3">
    <output name="out" type="color3" />
    <input name="in" type="color3" value="0, 0, 0" />
  </nodedef>
  <nodegraph name="NG_acescg_to_lin_rec709_color3" nodedef="ND_acescg_to_lin_rec709_color3">
    <convert name="asVec" type="vector3">
      <input name="in" type="color3" interfacename="in" />
    </convert>
    <transform name="matrixTransform" type="vector3">
      <input name="in" type="vector3" nodename="asVec" />
      <input name="mat" type="matrix33" value="1.7050509926579815, -0.6217921206570057, -0.08325887200097967, -0.1302564175070435, 1.140804736575405, -0.010548319068357662, -0.024003356804618046, -0.12896897606497093, 1.1529723328695858" />
    </transform>
    <convert name="asColor" type="color3">
      <input name="in" type="vector3" nodename="matrixTransform" />
    </convert>
    <output name="out" type="color3" nodename="asColor" />
  </nodegraph>
</materialx>
```
